In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import PIL
import tensorflow as tf
import pathlib

from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report


#  Data Loading

In [23]:
dataset_path = "./tea sickness dataset"
dataset_dir = pathlib.Path(dataset_path)

class_names = os.listdir(dataset_dir)
num_class = len(class_names)
image_files = [[os.path.join(dataset_dir, class_name, x) 
               for x in os.listdir(os.path.join(dataset_dir, class_name))] 
               for class_name in class_names]

In [24]:
image_file_list = []
image_label_list = []
for i, class_name in enumerate(class_names):
    image_file_list.extend(image_files[i])
    image_label_list.extend([i] * len(image_files[i]))
num_total = len(image_label_list)

In [25]:
image_width, image_height = Image.open(image_file_list[0]).size

print('Total image count:', num_total)
print("Image dimensions:", image_width, "x", image_height)
print("Label names:", class_names)
print("Label counts:", [len(image_files[i]) for i in range(num_class)])

Total image count: 885
Image dimensions: 768 x 1024
Label names: ['algal leaf', 'Anthracnose', 'bird eye spot', 'brown blight', 'gray light', 'healthy', 'red leaf spot', 'white spot']
Label counts: [113, 100, 100, 113, 100, 74, 143, 142]
